In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pwd
%cd /content/drive/My\ Drive/Verifiable-Coherent-NLU

/content
/content/drive/My Drive/Verifiable-Coherent-NLU


In [ ]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "Verifiable-Coherent-NLU"

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
import os
import sys

assert sys.version_info[0] == 3
assert sys.version_info[1] >= 5

GOOGLE_DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
sys.path.append(GOOGLE_DRIVE_PATH)

print(GOOGLE_DRIVE_PATH)


drive/My Drive/Verifiable-Coherent-NLU


In [ ]:
!pwd

/content/drive/My Drive/Verifiable-Coherent-NLU


In [ ]:
import argparse
import os
import random
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler, AutoTokenizer, AutoModelForMultipleChoice, RobertaForMultipleChoice
from datasets import load_dataset, load_metric, list_datasets
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
from tqdm.auto import tqdm
import evaluate

def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

SEED = 22
set_seed(SEED)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")


def load_data(tokenizer, params):

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #
    #                   TODO: Implementation                      #
    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #

    def tokenize_function(examples):
        observations = [[f"{observation_1} {examples['observation_2'][i]}"] * 2 for i, observation_1 in enumerate(examples["observation_1"])]

        hypotheses = [[hypothesis_1, examples["hypothesis_2"][i]] for i, hypothesis_1 in enumerate(examples["hypothesis_1"])]

        observations = sum(observations, [])
        hypotheses = sum(hypotheses, [])

        tokenized_examples = tokenizer(observations, hypotheses, truncation=True)
        tokenized_examples = {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

        tokenized_examples["answerKey"] = [label - 1 for label in examples["label"]]

        return tokenized_examples

        # questions = [[question] * 4 for question in examples["question"]]

        # choices_text = [choice["text"] for choice in examples["choices"]]

        # questions = sum(questions, [])
        # choices_text = sum(choices_text, [])

        # tokenized_examples = tokenizer(questions, choices_text, truncation=True)

        # tokenized_examples = {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

        # tokenized_examples["answerKey"] = [choice["label"].index(examples["answerKey"][i]) for i, choice in enumerate(examples["choices"])]

        # return tokenized_examples

    @dataclass
    class DataCollatorForMultipleChoice:
        """
        Data collator that will dynamically pad the inputs for multiple choice received.
        """

        tokenizer: PreTrainedTokenizerBase
        padding: Union[bool, str, PaddingStrategy] = True
        max_length: Optional[int] = None
        pad_to_multiple_of: Optional[int] = None

        def __call__(self, features):

            labels = [feature.pop('labels') for feature in features]
            batch_size = len(features)
            num_choices = len(features[0]["input_ids"])

            # Flatten
            flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
            flattened_features = sum(flattened_features, [])

            # Apply Padding
            batch = self.tokenizer.pad(
                flattened_features,
                padding=self.padding,
                max_length=self.max_length,
                pad_to_multiple_of=self.pad_to_multiple_of,
                return_tensors="pt",
            )

            # Un-flatten
            batch = {k: v.view(batch_size, num_choices, -1).to(device) for k, v in batch.items()}

            # Add back labels
            batch["labels"] = torch.tensor(labels, dtype=torch.int64).to(device)

            return batch

    # datasets_list = list_datasets()
    # print(', '.join(dataset.id for dataset in datasets_list))


    dataset = load_dataset('art.py')
    dataset = dataset.filter(lambda example : example["label"] != 0)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    tokenized_datasets = tokenized_datasets.rename_column("answerKey", "labels")

    accepted_keys = ["input_ids", "attention_mask", "labels"]
    for key in tokenized_datasets['train'].features.keys():
      if key not in accepted_keys:
        tokenized_datasets = tokenized_datasets.remove_columns(key)

    tokenized_datasets.set_format("torch")

    batch_size = params["batch_size"]
    data_collator = DataCollatorForMultipleChoice(tokenizer)

    train_dataset = tokenized_datasets["train"].shuffle(seed=SEED).select(range(2000))
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, collate_fn=data_collator)
    eval_dataset = tokenized_datasets["validation"]
    eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=data_collator)
    # test_dataset = tokenized_datasets["test"]
    # test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=data_collator)

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #

    # return train_dataloader, eval_dataloader, test_dataloader

    return train_dataloader, eval_dataloader


def finetune(model, train_dataloader, eval_dataloader, params):

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #
    #                   TODO: Implementation                      #
    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #
    num_training_steps = params["num_epochs"] * len(train_dataloader)
    optimizer = AdamW(model.parameters(), lr=params["lr"])
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    metric = evaluate.load("accuracy")
    progress_bar = tqdm(range(num_training_steps))

    for epoch in range(params["num_epochs"]):

        model.train()
        for batch in train_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

        model.eval()
        for batch in eval_dataloader:
            with torch.no_grad():
                outputs = model(**batch)

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            metric.add_batch(predictions=predictions, references=batch["labels"])

        score = metric.compute()
        print('Validation Accuracy:', score['accuracy'])
    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #

    return model


def test(model, test_dataloader, prediction_save='predictions.torch'):

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #
    #                   TODO: Implementation                      #
    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #

    metric = load_metric('accuracy')
    model.eval()
    all_predictions = []

    for batch in test_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    score = metric.compute()
    print('Test Accuracy:', score)
    torch.save(all_predictions, prediction_save)

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #

params = {"model": "bert-base-uncased",
      "batch_size": 10,
      "num_epochs": 10,
      "lr": 5e-5}

tokenizer = AutoTokenizer.from_pretrained(params["model"])
train_dataloader, eval_dataloader = load_data(tokenizer, params)

model = AutoModelForMultipleChoice.from_pretrained(params["model"])
model.to(device)
model = finetune(model, train_dataloader, eval_dataloader, params)

model.save_pretrained("/content/drive/My Drive/Verifiable-Coherent-NLU/art_finetuned/bs_10_lr_5e-5")
tokenizer.save_pretrained("/content/drive/My Drive/Verifiable-Coherent-NLU/art_finetuned/bs_10_lr_5e-5")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2000 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Validation Accuracy: 0.5202349869451697
Validation Accuracy: 0.5306788511749347
Validation Accuracy: 0.5300261096605744
Validation Accuracy: 0.5411227154046997
Validation Accuracy: 0.5339425587467362
Validation Accuracy: 0.5378590078328982
Validation Accuracy: 0.5365535248041775
Validation Accuracy: 0.5267624020887729
Validation Accuracy: 0.5267624020887729
Validation Accuracy: 0.5274151436031331


('/content/drive/My Drive/Verifiable-Coherent-NLU/art_finetuned/bs_10_lr_5e-5/tokenizer_config.json',
 '/content/drive/My Drive/Verifiable-Coherent-NLU/art_finetuned/bs_10_lr_5e-5/special_tokens_map.json',
 '/content/drive/My Drive/Verifiable-Coherent-NLU/art_finetuned/bs_10_lr_5e-5/vocab.txt',
 '/content/drive/My Drive/Verifiable-Coherent-NLU/art_finetuned/bs_10_lr_5e-5/added_tokens.json',
 '/content/drive/My Drive/Verifiable-Coherent-NLU/art_finetuned/bs_10_lr_5e-5/tokenizer.json')